In [8]:
import os
os.environ['autocnet_config'] = '/home/jlaura/autocnet_projects/demo.yml'
from autocnet_server.graph.graph import NetworkCandidateGraph

#Load the config file
with open(os.environ['autocnet_config'], 'r') as f:
    config = yaml.load(f)

In [9]:
ncg = NetworkCandidateGraph.from_database()

An AutoCNet server project configures 3 different redis queue using a containerized Redis server. In the below config, the server is running on `smalls` at port `8000`.  The three defined queues are:

* `processing_queue`: This is the queue that accepts messages from a job submitter.  New jobs are pushed onto this queue and wait for processing to begin.  When a job is pushed onto the `processing_queue` an associated cluster job has to be spawned.  Spawning this job is handled by the `NetworkCandidateGraph`.
* `working_queue`: When a cluster CPU becomes available and a job is started, the first thing that occurs is that the job is read from the processing queue and pushes the job to the working queue.  So far, we have worked very linearly waiting for one type of job to finish and then starting the next set of tasks.  Therefore, we have not mixed job types (e.g., extraction and matching) on a single queue.  If the ability to mix job types is desired, we can add it. The NetworkCandidateGraph watches the working_queue for jobs that have lingered for too long.  If a job exceed the walltime, the NetworkCandidateGraph removes the job message from the working queue and potentially resubmits the job with a different parameterization.
* `completed_queue`: If a job completes successfully, a message is written to the completed queue.  The completed queue is also watched by the NetworkCandidateGraph.  When a completed job is detected, the graph is updated in some way.  This update is generally exceptionally small (such as turning a 'did the job work' flag from False to True) as all data related operations are piped directly to the database (e.g., when new matches are found they are not piped through the NetworkCandidateGraph to the database rather, the cluster job writes them directly to the database.

In [3]:
config['redis']

{'completed_queue': 'jdemo:done',
 'host': 'smalls',
 'port': '8000',
 'processing_queue': 'jdemo:proc',
 'working_queue': 'jdemo:working'}

## Queue Flow
The queue flow seeks to:
- Never lose a job due to algorithm, network, or node failure
- Allow 'smarts' to reparameterize failed jobs a 'try again'

The order of operations is:

1. The NCG sends a message/messages to the processing queue and starts a job / jobarray
1. Each job pulls a message from the processing queue, takes the walltime of the job and computes the maximum time the job could run. This results in an updated message.
1. The updated message is pushed to the working queue
1. The NCG has a sentinel that watches the working queue for jobs that have lingered for more than the walltime.  If a job has lingered too long, it is removed from the working queue and resubmited to the processing queue
1. If a job is successful, the cluster process pushes a message to the completed queue and removes the task from the work queue

<img style="float: left;" src="../docs/images/queue_flow.png">




## Access the Queues
Redis queues are accessed by name.  Below, the three queue names are assigned to variables and the queues are queried for their total length.

The [PyRedis](https://redis-py.readthedocs.io/en/latest/#redis.StrictRedis) documentation is very good for describing how the queues can be interacted with.  In general, queues should be largely transparent to the user.

In [10]:
wq = config['redis']['working_queue']
pq = config['redis']['processing_queue']
cq = config['redis']['completed_queue']

In [12]:
print(ncg.redis_queue.llen(wq))
print(ncg.redis_queue.llen(pq))

0
72


Exception in thread jdemo:working:
Traceback (most recent call last):
  File "/home/jlaura/anaconda3/envs/ct/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/jlaura/autocnet_server/autocnet_server/graph/graph.py", line 525, in run
    self.queue.lrem(self.name,0, json.dumps(msg))
  File "/home/jlaura/autocnet_server/autocnet_server/graph/graph.py", line 360, in ring_matcher_callback
    if rm['count'] <= config['cluster']['maxfailures']:
KeyError: 'count'

